In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%pip install -U --pre triton
%pip install accelerate==0.12.0 transformers ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import requests
import tqdm
import os
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16).to("cuda")

# *****************************************************
instance_dir = "/content/drive/MyDrive/*******"
# *****************************************************

token = "xxx"
for filename in os.listdir(instance_dir):
  
  extension = filename.split(".")[-1]
  if extension != "txt":

    txtname = filename + ".txt"
    image = Image.open(instance_dir+"/"+filename).convert('RGB')

    text = ""
    inputs = processor(image, text, return_tensors="pt").to("cuda", torch.float16)

    out = model.generate(**inputs)
    out = processor.decode(out[0], skip_special_tokens=True)
    prompt = token + "," + out
    with open(instance_dir+"/"+txtname, 'w') as f:
          f.write(prompt)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


arafed image of a woman in a red sweater and a red sweater,in a ddzwn style
arafed image of a woman in a black shirt and jeans,in a ddzwn style
there is a woman in a white shirt and black suspender,in a ddzwn style
a woman sitting at a table with a cake and a cake,in a ddzwn style
smiling woman in pink sweater holding a cat in a room,in a ddzwn style
woman in red sweater and aprons cooking in a kitchen,in a ddzwn style
arafed image of a woman sitting on a bench in a library,in a ddzwn style
arafed image of a woman standing in a field with flowers,in a ddzwn style
a woman in a blue dress and umbrella hat is standing on a bridge,in a ddzwn style
a woman in a dress and a hat is sitting on a bench,in a ddzwn style
a woman in a dress and a purse is standing on a bridge,in a ddzwn style
arafed image of a woman standing on a bridge,in a ddzwn style
arafed image of a woman standing on a railing railing,in a ddzwn style
arafed image of a woman standing on a bridge over looking at the water,in a

In [ ]:
import ipywidgets as widgets
from io import BytesIO
#@markdown #Captions

#@markdown - Open a tool to manually `create` captions or edit existing captions of the instance images.

paths=""
out=""
widgets_l=""

instance_dir = 0 #@param
DIR = instance_dir
def Caption(path):
  
    if path!="Select an instance image to caption":
      
      name = os.path.basename(path)
      ext=os.path.splitext(os.path.basename(path))[-1][1:]
      if ext=="jpg" or "JPG":
        ext="JPEG"
      else:
        ext="PNG"      

      if os.path.exists(DIR + "/"+ name + '.txt'):
        with open(DIR + "/" + name + '.txt', 'r') as f:
            text = f.read()
      else:
        with open(DIR + "/" + name + '.txt', 'w') as f:
            f.write("")
            with open(DIR + "/" + name + '.txt', 'r') as f:
                text = f.read()   

      img=Image.open(os.path.join(DIR,path))
      img = img.convert('RGB').resize((420, 420))
      image_bytes = BytesIO()
      img.save(image_bytes, format=ext, qualiy=10)
      image_bytes.seek(0)
      image_data = image_bytes.read()
      img= image_data
      image = widgets.Image(
          value=img,
          width=420,
          height=420
      )
      text_area = widgets.Textarea(value=text, description='', disabled=False, layout={'width': '500px', 'height': '120px'})
      

      def update_text(text):
          with open(DIR+"/"+ name + '.txt', 'w') as f:
              f.write(text)

      button = widgets.Button(description='Save', button_style='success')
      button.on_click(lambda b: update_text(text_area.value))
      
      # return widgets.VBox([widgets.HBox([text_area, button])])
      return widgets.HBox([image, widgets.VBox([text_area, button])])


paths = os.listdir(DIR)
widgets_l = widgets.Select(options=["Select an instance image to caption"]+paths, rows=25)


out = widgets.Output()

def click(change):
    with out:
        out.clear_output()
        display(Caption(change.new))

widgets_l.observe(click, names='value')
display(widgets.HBox([widgets_l, out]))